Imports

In [270]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

[[1,2,3,4,5,6,7], [1,2,3,4,5,6,7]]

In [271]:
ds = pd.read_csv('falls_vs_normal.csv')
ds

,xAcc,yAcc,zAcc,xGyro,yGyro,zGyro,label
0,6.99,-0.57,-7.28,-2.75,-3.23,2.62,fall
1,6.51,-0.75,-6.93,-0.67,-6.35,4.64,fall
2,6.22,-0.63,-6.73,0.79,-5.49,3.85,fall
3,6.34,-0.62,-6.80,1.59,-2.26,0.67,fall
4,6.49,-0.39,-6.60,0.67,-0.24,-1.10,fall
...,...,...,...,...,...,...,...
96795,7.30,-1.16,-4.58,18.19,3.60,-35.77,light
96796,7.39,-0.37,-5.05,20.08,0.06,-34.55,light
96797,7.52,-1.46,-5.82,22.58,0.12,-28.02,light
96798,7.58,-2.14,-6.31,19.47,2.44,-22.52,light


In [272]:

#def split_observations(ds, window_size):
  ds_arr = ds.to_numpy()
  observations = []
  y_values = []

  for i in range(0, len(ds_arr), window_size):
    observation = ds_arr[i:i + window_size, :-1]  # Slice a window, exclude last col
    observations.append(observation)

    # Append y value every window_size (i.e., every 400 values)
    if i % 400 == 0:  # Check for divisibility by 400, not remainder
      y_values.append(ds_arr[i, -1])  # Append only the last value of the window

  return np.array(observations), np.array(y_values)
        

In [274]:
WINDOW_SIZE = 400
x, y = split_observations(ds, WINDOW_SIZE)
x.shape, y.shape

ValueError: too many values to unpack (expected 2)

In [ ]:
X_train, y_train = x[:200], y[:200]
X_val, y_val = x[200:220], y[200:220]
X_test, y_test = x[220:], y[220:]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape
print(X_train[0])
print(y_train[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming y_train contains string labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)

# Reshape y_train (and similarly y_val, y_test) for multi-class classification
y_train = y_train.reshape(y_train.shape[0], 1, -1)
y_val = y_val.reshape(y_val.shape[0], 1, -1)
y_test = y_test.reshape(y_test.shape[0], 1, -1)
y_train.shape, y_val.shape, y_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
model1 = Sequential()
model1.add(InputLayer((400,6)))
model1.add(LSTM(256))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(1, activation='linear'))
model1.summary()

In [ ]:
cp = ModelCheckpoint('model2/model.keras', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [ ]:
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks=[cp])